# IMDB Sentiment Analysis using LSTM

## Importing libraries

In [14]:
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split as tts
from keras.layers import Dense, LSTM,Embedding, SpatialDropout1D
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import roc_auc_score
import tensorflow as tf


<function tensorflow.python.autograph.impl.api.do_not_convert(func=None)>

## Reading the dataset

## Text Preprocessing

In [8]:
import string
# removing the html tags
def clean_html(text):
    clean=re.compile('<.*?>')
    cleantext=re.sub(clean,'',text)
    return cleantext
    
# first round of cleaning
def clean_text1(text):
    text=text.lower()
    text=re.sub('\[.*?\]','',text)
    text=re.sub('[%s]'%re.escape(string.punctuation),'',text)
    text=re.sub('\w*\d\w*','',text)
    return text

# second round of cleaning
def clean_text2(text):
    text=re.sub('[''"",,,]','',text)
    text=re.sub('\n','',text)
    return text
    
cleaned_html=lambda x:clean_html(x)
cleaned1=lambda x:clean_text1(x)
cleaned2=lambda x:clean_text2(x)

## Defining the model

In [23]:
res = []
for i in range(1,6):
    print(f'>>>>> Fold:{i} <<<<<')
    folderName = 'split_' + str(i)
    train_filename = folderName + '/' + 'train.tsv'
    test_filename = folderName + '/' + 'test.tsv'
    test_y_filename = folderName + '/' + 'test_y.tsv'

    train_data = pd.read_csv(train_filename,sep='\t', header=0)
    train_data['review']=pd.DataFrame(train_data.review.apply(cleaned_html))
    train_data['review']=pd.DataFrame(train_data.review.apply(cleaned1))
    train_data['review']=pd.DataFrame(train_data.review.apply(cleaned2))

    train_y = train_data['sentiment']
    train_features = train_data.copy()
    train_features = train_features.drop(['sentiment'],axis=1)

    test_data = pd.read_csv(test_filename,sep='\t', header=0)
    test_data['review']=pd.DataFrame(test_data.review.apply(cleaned_html))
    test_data['review']=pd.DataFrame(test_data.review.apply(cleaned1))
    test_data['review']=pd.DataFrame(test_data.review.apply(cleaned2))

    test_features = test_data['review']
    # print(test_data.head(5))
    # print(test_data.shape)
    test_y_data = pd.read_csv(test_y_filename,sep='\t', header=0)

    max_features=5000
    tokenizer = Tokenizer(num_words=max_features, split=' ')
    tokenizer.fit_on_texts(train_data['review'].values)

    X_train = tokenizer.texts_to_sequences(train_data['review'].values)
    X_train = pad_sequences(X_train,maxlen=600)
    Y_train = pd.get_dummies(train_data['sentiment']).values
    # Y_train = train_data['sentiment']

    tokenizer.fit_on_texts(test_data['review'].values)
    X_test = tokenizer.texts_to_sequences(test_data['review'].values)
    X_test = pad_sequences(X_test,maxlen=600)
    Y_test = pd.get_dummies(test_y_data['sentiment']).values
    # Y_test = test_y_data['sentiment']

    embed_dim = 128
    lstm_out = 128
    
    model = Sequential()
    model.add(Embedding(max_features, embed_dim,input_length = X_train.shape[1]))
    model.add(SpatialDropout1D(0.4))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(2,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    batch_size = 64

    # X_train = tf.autograph.experimental.do_not_convert(X_train)
    # Y_train = tf.autograph.experimental.do_not_convert(Y_train)

    # X_test = tf.autograph.experimental.do_not_convert(X_test)
    # Y_test = tf.autograph.experimental.do_not_convert(Y_test)
    # model = tf.autograph.experimental.do_not_convert(model)
    model.fit(X_train, Y_train, epochs = 10, batch_size=batch_size,validation_data=(X_test,Y_test),verbose=True)
    pred_Y = model.predict(X_test)
    auc = roc_auc_score(Y_test, pred_Y,average='micro')
    res.append(auc)
    print('>>>>> Running is done <<<<<')
print('>>>>> All folds are done <<<<<')

>>>>> Fold:1 <<<<<
Epoch 1/10
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
391/391 [==============================] - ETA: 0s - loss: 0.4492 - accuracy: 0.7869WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x7f78949a0840> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAP

In [24]:

mean_auc = np.mean(res)

print(f'mean AUC:{mean_auc}')

mean AUC:0.60129185312


In [25]:
model2 = Sequential()
model2.add(Embedding(max_features, embed_dim,input_length = X_train.shape[1]))
model2.add(SpatialDropout1D(0.4))
model2.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model2.add(Dense(2,activation='softmax'))
model2.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
batch_size = 64

In [27]:
res2 = []
for i in range(1,6):
    print(f'>>>>> Fold:{i} <<<<<')
    folderName = 'split_' + str(i)
    train_filename = folderName + '/' + 'train.tsv'
    test_filename = folderName + '/' + 'test.tsv'
    test_y_filename = folderName + '/' + 'test_y.tsv'

    train_data = pd.read_csv(train_filename,sep='\t', header=0)
    train_data['review']=pd.DataFrame(train_data.review.apply(cleaned_html))
    train_data['review']=pd.DataFrame(train_data.review.apply(cleaned1))
    train_data['review']=pd.DataFrame(train_data.review.apply(cleaned2))

    train_y = train_data['sentiment']
    train_features = train_data.copy()
    train_features = train_features.drop(['sentiment'],axis=1)

    test_data = pd.read_csv(test_filename,sep='\t', header=0)
    test_data['review']=pd.DataFrame(test_data.review.apply(cleaned_html))
    test_data['review']=pd.DataFrame(test_data.review.apply(cleaned1))
    test_data['review']=pd.DataFrame(test_data.review.apply(cleaned2))

    test_features = test_data['review']
    # print(test_data.head(5))
    # print(test_data.shape)
    test_y_data = pd.read_csv(test_y_filename,sep='\t', header=0)

    max_features=5000
    tokenizer = Tokenizer(num_words=max_features, split=' ')
    tokenizer.fit_on_texts(train_data['review'].values)

    X_train = tokenizer.texts_to_sequences(train_data['review'].values)
    X_train = pad_sequences(X_train,maxlen=600)
    Y_train = pd.get_dummies(train_data['sentiment']).values
    # Y_train = train_data['sentiment']

    tokenizer.fit_on_texts(test_data['review'].values)
    X_test = tokenizer.texts_to_sequences(test_data['review'].values)
    X_test = pad_sequences(X_test,maxlen=600)
    Y_test = pd.get_dummies(test_y_data['sentiment']).values
    # Y_test = test_y_data['sentiment']

    embed_dim = 128
    lstm_out = 128
    
    model2.fit(X_train, Y_train, epochs = 10, batch_size=batch_size,validation_data=(X_test,Y_test),verbose=True)
    pred_Y = model2.predict(X_test)
    auc = roc_auc_score(Y_test, pred_Y,average='micro')
    res2.append(auc)
    print('>>>>> Running is done <<<<<')
print('>>>>> All folds are done <<<<<')

>>>>> Fold:1 <<<<<
Epoch 1/10
391/391 [==============================] - ETA: 0s - loss: 0.5386 - accuracy: 0.7438WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x7f78949a0d08> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
391/391 [==============================] - 382s 977ms/step - loss: 0.5386 - accuracy: 0.7438 - val_loss: 0.7557 - v

In [28]:
mean_auc2 = np.mean(res2)

print(f'mean AUC:{mean_auc2}')

mean AUC:0.61725748928
